<a href="https://colab.research.google.com/github/Anson3208/Fine-Tune-GPT/blob/main/Fine_tune_GPT_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deployment of fine-tune model

The fine-tune model will be deployed in Google Cloud. 

Given that the primary focus of the project is GPT fine-tuning, the web development aspect involving HTML and CSS will be implemented with minimal refinement and simplicity.

# Google Cloud setup

The code performs Google Cloud authentication and sets the project configuration for deployment.

In [1]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=4m2aOUQfsp0Z27rmiB1RbPgAaoAcnR&prompt=consent&access_type=offline&code_challenge=Sx4-1bg9GdoWoOoyJBPavW9Fr-6hxSBGkRJGNTN5dZE&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VMX_6JTfunWWJkwO1co_uMVayZdG8ucuFBuZjfTUMvZ0p2aC4HIRpq7rsFCxQ36nQ

You are now logged in as [ansonchan3208@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config set project my-project-anson-382915

Updated property [core/project].


In [3]:
!mkdir finetunegpt

In [4]:
from google.colab import files
uploaded_files = files.upload()

Saving api_key.txt to api_key.txt


# Creating the Fine-tune GPT Docker Container

The code creates a Dockerfile and sets up the necessary configurations for the Fine-tune GPT model.

In [5]:
!mv api_key.txt finetunegpt      # move apikey into finetunegpt folder

In [6]:
with open('finetunegpt/Dockerfile','w') as f:
  f.write('''
# Use the official lightweight Python image.
# https://hub.docker.com/_/python
FROM python:3.10-slim

# Allow statements and log messages to immediately appear in the Knative logs
ENV PYTHONUNBUFFERED True

# Copy local code to the container image.
ENV APP_HOME /app
WORKDIR $APP_HOME
COPY . ./

# Install production dependencies.
RUN pip install --no-cache-dir -r requirements.txt

# Run the web service on container startup. Here we use the gunicorn
# webserver, with one worker process and 8 threads.
# For environments with multiple CPU cores, increase the number of workers
# to be equal to the cores available.
# Timeout is set to 0 to disable the timeouts of the workers to allow Cloud Run to handle instance scaling.
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 main:app
  ''')

In [7]:
with open('finetunegpt/.dockerignore','w') as f:
  f.write('''
Dockerfile
README.md
*.pyc
*.pyo
*.pyd
__pycache__
.pytest_cache
  ''')

In [8]:
with open('finetunegpt/requirements.txt','w') as f:
  f.write('''
Flask==2.1.0
gunicorn==20.1.0
openai==0.27.4
  ''')

In [9]:
with open('finetunegpt/main.py','w') as f:
  f.write('''
import os
from flask import Flask, request, session, render_template
import openai
import secrets

with open('api_key.txt') as f:
  api_key = f.read()
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = openai.api_key

app = Flask(__name__)
app.secret_key = secrets.token_hex(16)

@app.route("/", methods=['GET', 'POST'])
def hello_world():
    if request.method == 'POST':
        user_input = request.form['user_input']
        chat_log = session.get('chat_log', [['','']])
        chat_log.extend([['user',user_input]])
        response = openai.Completion.create(
            model="ada:ft-personal:personality-2023-05-23-16-30-52",
            prompt=user_input,
            temperature=0.7,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["END"]
        )
        chat_log.extend([['gpt',response['choices'][0]['text']]])
        session['chat_log'] = chat_log
    else:
        session['chat_log'] = [['','']]
    return render_template('index.html', chat_log=session['chat_log'])


if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))
  ''')

# Creating the HTML Template for the Fine-tune GPT Web Interface

The code creates an HTML template that serves as the interface for the Fine-tune GPT model.

In [10]:
!mkdir finetunegpt/templates

In [11]:
with open('finetunegpt/templates/index.html','w') as f:
  f.write('''
<!doctype html>
<html>
  <head>
    <title>Fine Tune GPT</title>
  </head>
  <body>
    <h1>Welcome! </h1>
    <p> This is a showcase of the results from my first Fine-tuned GPT project.</p><br>
    <p> The Fine-tuned GPT model is capable of generating descriptions based on prompts such as Age Description, Gender, and Personality. 
    However, please note that for this demonstration, a more cost-effective model named Ada is being utilized, which may result in responses of lower quality</p><br>
    <p>Example: Young, Man, Energetic-></p>
    <form method="POST" action="/">
      <input style='width:550px' type="text" name="user_input">
      <button style='width:50px' type="submit">Send</button>
    </form>
    {% for i in chat_log %}
      {% if i[0]=='user' %}
      <p style='width:600px'><em>{{i[1]}}</em></p>
      {% else %}
      <p style='width:600px'>{{i[1]}}</p>
      {% endif %}
    {% endfor %}
    <p>For detail coding, please check out my github:</p>
    <a href="https://github.com/Anson3208/Fine-Tune-GPT">https://github.com/Anson3208/Fine-Tune-GPT</a>
  </body>
</html>
  ''')

In [12]:
!gcloud run deploy

Deploying from source. To deploy a container use [--image]. See https://cloud.google.com/run/docs/deploying-source-code for more details.
Source code location (/content):  finetunegpt
Next time, use `gcloud run deploy --source .` to deploy the current directory.

Service name (finetunegpt):  finetunegpt
Please specify a region:
 [1] asia-east1
 [2] asia-east2
 [3] asia-northeast1
 [4] asia-northeast2
 [5] asia-northeast3
 [6] asia-south1
 [7] asia-south2
 [8] asia-southeast1
 [9] asia-southeast2
 [10] australia-southeast1
 [11] australia-southeast2
 [12] europe-central2
 [13] europe-north1
 [14] europe-southwest1
 [15] europe-west1
 [16] europe-west12
 [17] europe-west2
 [18] europe-west3
 [19] europe-west4
 [20] europe-west6
 [21] europe-west8
 [22] europe-west9
 [23] me-central1
 [24] me-west1
 [25] northamerica-northeast1
 [26] northamerica-northeast2
 [27] southamerica-east1
 [28] southamerica-west1
 [29] us-central1
 [30] us-east1
 [31] us-east4
 [32] us-east5
 [33] us-south1
 [34